### 1. SETUP DAN PERSIAPAN DATA

In [1]:
import os
import sys
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
import shutil
import subprocess
import requests
import time
import grpc
from tensorflow_serving.apis import prediction_service_pb2_grpc
from tensorflow_serving.apis.predict_pb2 import PredictRequest

2025-06-20 23:48:23.572487: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750438103.649044  269604 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750438103.670618  269604 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-20 23:48:23.804333: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### 2. MEMBUAT DAN MELATIH MODEL AWAL (VERSI 1) 

In [2]:
# Memastikan output stabil di setiap eksekusi dengan mengatur random seed
np.random.seed(42)
tf.random.set_seed(42)

# Memuat dataset MNIST
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.mnist.load_data()

# Normalisasi dan penambahan dimensi channel pada data gambar
X_train_full = X_train_full[..., np.newaxis].astype(np.float32) / 255.
X_test = X_test[..., np.newaxis].astype(np.float32) / 255.

# Membagi data training menjadi set validasi dan training
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

# Mengambil 3 sampel pertama dari data test untuk prediksi
X_new = X_test[:3]

print("Setup dan persiapan data selesai.")
print(f"Bentuk X_train: {X_train.shape}")
print(f"Bentuk X_new untuk tes: {X_new.shape}\n")

Setup dan persiapan data selesai.
Bentuk X_train: (55000, 28, 28, 1)
Bentuk X_new untuk tes: (3, 28, 28, 1)



In [3]:
# Functional API — model stabil dan bisa disimpan di SavedModel
inputs = keras.Input(shape=(28, 28, 1))
x = keras.layers.Flatten()(inputs)
x = keras.layers.Dense(100, activation="relu")(x)
outputs = keras.layers.Dense(10, activation="softmax")(x)

model = keras.Model(inputs, outputs)

# Compile & train
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-2),
              metrics=["accuracy"])
model.fit(X_train, y_train, epochs=5, validation_data=(X_valid, y_valid))

I0000 00:00:1750438114.312566  269604 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4057 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5
W0000 00:00:1750438114.857594  270148 gpu_backend_lib.cc:579] Can't find libdevice directory ${CUDA_DIR}/nvvm/libdevice. This may result in compilation or runtime failures, if the program we try to run uses routines from libdevice.
Searched for CUDA in the following directories:
  ./cuda_sdk_lib
  ipykernel_launcher.runfiles/cuda_nvcc
  ipykern/cuda_nvcc
  
  /usr/local/cuda
  /home/ardi/miniconda3/lib/python3.12/site-packages/tensorflow/python/platform/../../../nvidia/cuda_nvcc
  /home/ardi/miniconda3/lib/python3.12/site-packages/tensorflow/python/platform/../../../../nvidia/cuda_nvcc
  /home/ardi/miniconda3/lib/python3.12/site-packages/tensorflow/python/platform/../../cuda
  .
You can choose the search directory by setting xla_gpu_cuda_data_dir in HloMo

Epoch 1/5


I0000 00:00:1750438117.078515  270142 service.cc:148] XLA service 0x7fd9fc004b70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1750438117.079041  270142 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 2060, Compute Capability 7.5
2025-06-20 23:48:37.115385: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1750438117.185948  270142 cuda_dnn.cc:529] Loaded cuDNN version 91000


  22/1719 ━━━━━━━━━━━━━━━━━━━━ 13s 8ms/step - accuracy: 0.1722 - loss: 2.2798

I0000 00:00:1750438118.186260  270142 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1719/1719 ━━━━━━━━━━━━━━━━━━━━ 18s 9ms/step - accuracy: 0.7299 - loss: 1.0653 - val_accuracy: 0.9030 - val_loss: 0.3678
Epoch 2/5
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.8976 - loss: 0.3670 - val_accuracy: 0.9194 - val_loss: 0.2950
Epoch 3/5
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.9147 - loss: 0.3047 - val_accuracy: 0.9296 - val_loss: 0.2599
Epoch 4/5
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.9247 - loss: 0.2702 - val_accuracy: 0.9358 - val_loss: 0.2362
Epoch 5/5
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.9310 - loss: 0.2456 - val_accuracy: 0.9400 - val_loss: 0.2182


### 3. MENYIMPAN MODEL DALAM FORMAT SAVEDMODEL 

In [4]:
# Buat folder model jika belum ada
model_dir = "my_mnist_model"
model_name = "my_mnist_model"
os.makedirs(model_dir, exist_ok=True)

tf.saved_model.save(model, model_dir)
print(f"Model berhasil disimpan dalam format SavedModel di: {model_dir}")

print("\n-- Menampilkan signature dari SavedModel secara langsung tanpa CLI --")
model = tf.saved_model.load(model_dir)
signature = loaded_model.signatures["serving_default"]
print("Input Signature:")
print(signature.structured_input_signature)
print("Output Signature:")
print(signature.structured_outputs)

# Menampilkan struktur direktori dari SavedModel
print("\nStruktur direktori SavedModel yang dihasilkan:")
for root, dirs, files in os.walk(model_name):
    indent = '    ' * root.count(os.sep)
    print('{}{}/'.format(indent, os.path.basename(root)))
    for filename in files:
        print('{}{}'.format(indent + '    ', filename))

# Menggunakan saved_model_cli untuk menginspeksi model
print("\nInspeksi model menggunakan saved_model_cli:")
try:
    print("\n-- Menampilkan semua tag set dan signature defs --")
    # Di lingkungan non-shell, lebih aman menggunakan list daripada string
    print("\n-- Menampilkan signature dari SavedModel secara langsung tanpa CLI --")
    loaded_model = tf.saved_model.load(model_dir)
    signature = loaded_model.signatures["serving_default"]
    print("Input Signature:")
    print(signature.structured_input_signature)
    print("Output Signature:")
    print(signature.structured_outputs)
    
    print("\n-- Menampilkan detail signature 'serving_default' --")
    subprocess.run(
        ["saved_model_cli", "show", "--dir", model_dir, "--tag_set", "serve", "--signature_def", "serving_default"],
        check=True
    )
except (subprocess.CalledProcessError, FileNotFoundError):
    print("Peringatan: `saved_model_cli` tidak ditemukan. Lewati inspeksi CLI.")

print("\nPenyimpanan model selesai.\n")

TypeError: this __dict__ descriptor does not support '_DictWrapper' objects

### 4. MENJALANKAN TENSORFLOW SERVING & MENGIRIM REQUEST 

In [5]:
# TensorFlow Serving harus diinstal. Anda bisa menggunakan Docker (rekomendasi) atau
# menginstalnya langsung:
#
# Opsi 1 (Docker):
# docker run -it --rm -p 8500:8500 -p 8501:8501 \
#    -v "$(pwd)/my_mnist_model:/models/my_mnist_model" \
#    -e MODEL_NAME=my_mnist_model \
#    tensorflow/serving
#
# Opsi 2 (Instalasi Lokal):
# apt-get install tensorflow-model-server
#
# Skrip ini akan mencoba menjalankan `tensorflow_model_server` di background.
# Pastikan `tensorflow-model-server` terinstal dan bisa diakses dari terminal.

model_dir = os.path.abspath(os.path.join(model_dir, ".."))
server_proc = None
model_name = os.path.basename(model_dir)
try:
    # Menjalankan TensorFlow Model Server sebagai proses background
    print("Mencoba menjalankan TensorFlow Model Server di background...")
    command = [
        "tensorflow_model_server",
        "--rest_api_port=8501",
        "--port=8500",
        f"--model_name={model_name}",
        f"--model_base_path={model_dir}"
    ]
    
    # Menggunakan Popen untuk menjalankan server di background
    server_proc = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print(f"Server berjalan dengan PID: {server_proc.pid}")
    
    # Beri waktu beberapa detik agar server siap
    print("Menunggu server siap (10 detik)...")
    time.sleep(10)

    # --- 4.1 Menggunakan REST API ---
    print("\n--- 4.1 Mengirim request via REST API ---")
    
    # Menyiapkan data request dalam format JSON
    input_data_json = json.dumps({
        "signature_name": "serving_default",
        "instances": X_new.tolist(),
    })
    
    # URL untuk endpoint predict REST API
    SERVER_URL = f'http://localhost:8501/v1/models/{model_name}:predict'
    
    # Mengirim request POST
    response = requests.post(SERVER_URL, data=input_data_json)
    response.raise_for_status() # Akan error jika status code bukan 2xx
    
    predictions = response.json()['predictions']
    y_proba_rest = np.array(predictions)
    
    print("Hasil prediksi dari REST API:")
    print(y_proba_rest.round(2))

    # --- 4.2 Menggunakan gRPC API ---
    print("\n--- 4.2 Mengirim request via gRPC API ---")
    
    # Menyiapkan request gRPC
    request_grpc = PredictRequest()
    request_grpc.model_spec.name = model_name
    request_grpc.model_spec.signature_name = "serving_default"
    input_name = model.input_names[0]
    request_grpc.inputs[input_name].CopyFrom(tf.make_tensor_proto(X_new))
    
    # Membuat channel dan stub gRPC
    channel = grpc.insecure_channel('localhost:8500')
    predict_service = prediction_service_pb2_grpc.PredictionServiceStub(channel)
    
    # Mengirim request gRPC
    response_grpc = predict_service.Predict(request_grpc, timeout=10.0)
    
    # Mengonversi hasil gRPC (protobuf) kembali ke NumPy array
    output_name = model.output_names[0]
    outputs_proto = response_grpc.outputs[output_name]
    y_proba_grpc = tf.make_ndarray(outputs_proto) # Cara mudah jika TF terinstall
    
    print("Hasil prediksi dari gRPC API:")
    print(y_proba_grpc.round(2))
    
except (FileNotFoundError, requests.exceptions.ConnectionError) as e:
    print(f"\nERROR: Tidak dapat menjalankan atau terhubung ke TensorFlow Model Server.")
    print("Pastikan `tensorflow_model_server` terinstal dan dapat diakses, atau jalankan via Docker.")
    print(f"Detail error: {e}")
except Exception as e:
    print(f"\nTerjadi error: {e}")
finally:
    # Memastikan server dihentikan setelah selesai
    if server_proc:
        print("\n\nMenghentikan TensorFlow Model Server...")
        server_proc.terminate()
        server_proc.wait()
        print("Server dihentikan.")

print("\nDeploy dan query model versi 1 selesai.\n")

Mencoba menjalankan TensorFlow Model Server di background...

ERROR: Tidak dapat menjalankan atau terhubung ke TensorFlow Model Server.
Pastikan `tensorflow_model_server` terinstal dan dapat diakses, atau jalankan via Docker.
Detail error: [Errno 2] No such file or directory: 'tensorflow_model_server'

Deploy dan query model versi 1 selesai.



### 5. DEPLOY VERSI MODEL BARU (VERSI 2)  

In [6]:
import time
import requests
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Pastikan variabel dari sel sebelumnya tersedia (X_train, y_train, dll.)
# dan server untuk v1 sedang berjalan.

# 1. MEMBUAT DAN MELATIH MODEL VERSI 2
# =====================================

# Membuat model baru dengan arsitektur berbeda
model_v2 = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]), # Menggunakan input shape 28x28 untuk MNIST
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

# Mengompilasi model v2
model_v2.compile(loss="sparse_categorical_crossentropy",
                 optimizer=keras.optimizers.SGD(learning_rate=1e-2),
                 metrics=["accuracy"])

print("\nMelatih model versi 2...")
# Asumsi X_train, y_train, X_valid, y_valid sudah ada dari sel sebelumnya
history_v2 = model_v2.fit(X_train, y_train, epochs=5, validation_data=(X_valid, y_valid), verbose=1)


# 2. MENYIMPAN DAN MENDEPLOY MODEL VERSI 2 SECARA OTOMATIS
# ========================================================

# Menentukan path untuk model versi 2
model_version_v2 = "0002"
model_path_v2 = os.path.join(model_name, model_version_v2) # model_name dari sel sebelumnya
print(f"\nMenyimpan model versi 2 ke path: {model_path_v2}")
tf.saved_model.save(model_v2, model_path_v2)

print("\nMenunggu TensorFlow Serving (yang sedang berjalan) memuat model versi 2...")
# Beri waktu sekitar 10-15 detik agar TFS mendeteksi folder model baru
time.sleep(15)

# 3. MENGIRIM PERMINTAAN PREDIKSI KE SERVER
# ============================================
# Kita tidak perlu memulai server baru. Cukup kirim request ke URL server yang sama.
# TFS akan secara otomatis menggunakan versi model terbaru (0002).

try:
    print("\nMengirim request lagi ke server (seharusnya sudah menggunakan model v2)...")
    
    # Asumsi SERVER_URL dan input_data_json sudah ada dari sel sebelumnya
    response_v2 = requests.post(SERVER_URL, data=input_data_json)
    response_v2.raise_for_status()
    
    predictions_v2 = response_v2.json()['predictions']
    y_proba_v2 = np.array(predictions_v2)
    
    print("Hasil prediksi dari model versi 2:")
    print(y_proba_v2.round(2))
    
    # Membandingkan dengan prediksi dari model pertama (jika ada)
    # print("\nHasil prediksi dari model versi 1 (sebelumnya):")
    # print(y_proba_v1.round(2)) # y_proba_v1 dari sel sebelumnya

except (NameError, requests.exceptions.RequestException) as e:
    print(f"\nERROR: Gagal mengirim permintaan ke TensorFlow Model Server.")
    print("Pastikan variabel SERVER_URL dan input_data_json sudah ada, dan server sedang berjalan.")
    print(f"Detail error: {e}")

print("\nDemonstrasi deploy model versi baru selesai.\n")
# Jangan terminate server di sini agar bisa digunakan di sel berikutnya jika perlu.
# Proses server utama akan dihentikan di akhir notebook.

/home/ardi/miniconda3/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



Melatih model versi 2...
Epoch 1/5
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - accuracy: 0.6493 - loss: 1.1883 - val_accuracy: 0.9060 - val_loss: 0.3444
Epoch 2/5
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.9011 - loss: 0.3455 - val_accuracy: 0.9238 - val_loss: 0.2790
Epoch 3/5
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 13s 8ms/step - accuracy: 0.9179 - loss: 0.2864 - val_accuracy: 0.9310 - val_loss: 0.2449
Epoch 4/5
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.9285 - loss: 0.2495 - val_accuracy: 0.9394 - val_loss: 0.2194
Epoch 5/5
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.9358 - loss: 0.2218 - val_accuracy: 0.9436 - val_loss: 0.2000

Menyimpan model versi 2 ke path: Deep-Learning/0002


TypeError: this __dict__ descriptor does not support '_DictWrapper' objects

### 6. (OPSIONAL) DEPLOY KE GOOGLE CLOUD AI PLATFORM

In [ ]:
# Bagian ini dinonaktifkan karena memerlukan setup spesifik di Google Cloud.
# Anda perlu memiliki project_id dan file kredensial JSON.
# Jika Anda ingin menjalankannya, hapus komentar dan pastikan
# pustaka `google-api-python-client` terinstal.

# print("=== BAGIAN 6: DEPLOY KE GOOGLE CLOUD AI PLATFORM (Dinonaktifkan) ===")
# project_id = "your-gcp-project-id"
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "my_service_account_private_key.json"
#
# if os.path.exists(os.environ["GOOGLE_APPLICATION_CREDENTIALS"]):
#     import googleapiclient.discovery
#
#     model_id = "my_mnist_model"
#     model_path_gcp = "projects/{}/models/{}".format(project_id, model_id)
#     ml_resource = googleapiclient.discovery.build("ml", "v1").projects()
#
#     def predict_gcp(X):
#         input_data_json = {"signature_name": "serving_default",
#                            "instances": X.tolist()}
#         request = ml_resource.predict(name=model_path_gcp, body=input_data_json)
#         response = request.execute()
#         if "error" in response:
#             raise RuntimeError(response["error"])
#         output_name = model.output_names[0] # Mengambil nama output dari model
#         return np.array([pred[output_name] for pred in response["predictions"]])
#
#     try:
#         print("Mengirim request ke Google Cloud AI Platform...")
#         Y_probas_gcp = predict_gcp(X_new)
#         print("Hasil prediksi dari GCP:")
#         print(np.round(Y_probas_gcp, 2))
#     except Exception as e:
#         print(f"Gagal melakukan prediksi di GCP: {e}")
# else:
#     print("File kredensial GCP tidak ditemukan. Melewati bagian ini.")

### 7. PENGECEKAN GPU

In [7]:
# Kode ini telah diperbarui untuk menggunakan API modern TensorFlow 2.x
# `tf.test.is_gpu_available()` sudah usang.

# Cara modern untuk memeriksa ketersediaan GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"GPU terdeteksi: {len(gpus)} buah")
    for gpu in gpus:
        print(f"  - {gpu}")
    # Menampilkan nama perangkat GPU utama (jika ada)
    # `tf.test.gpu_device_name()` masih berfungsi namun tidak direkomendasikan
    try:
        print(f"Nama perangkat GPU utama: {tf.test.gpu_device_name()}")
    except:
        print("Tidak bisa mendapatkan nama perangkat GPU utama via tf.test.gpu_device_name().")

else:
    print("Tidak ada GPU yang terdeteksi.")

# Memeriksa apakah TensorFlow di-build dengan dukungan CUDA
print(f"Apakah TensorFlow di-build dengan CUDA? {'Ya' if tf.test.is_built_with_cuda() else 'Tidak'}")

# Menampilkan semua perangkat lokal yang terdeteksi oleh TensorFlow (CPU & GPU)
from tensorflow.python.client import device_lib
print("\nDaftar semua perangkat lokal yang terdeteksi:")
devices = device_lib.list_local_devices()
for device in devices:
    print(f"- Tipe: {device.device_type}, Nama: {device.name}, Memori: {device.memory_limit // (1024*1024)} MB")

GPU terdeteksi: 1 buah
  - PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
Nama perangkat GPU utama: /device:GPU:0
Apakah TensorFlow di-build dengan CUDA? Ya

Daftar semua perangkat lokal yang terdeteksi:
- Tipe: CPU, Nama: /device:CPU:0, Memori: 256 MB
- Tipe: GPU, Nama: /device:GPU:0, Memori: 4057 MB


I0000 00:00:1750438361.389138  269604 gpu_device.cc:2022] Created device /device:GPU:0 with 4057 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5
I0000 00:00:1750438361.390961  269604 gpu_device.cc:2022] Created device /device:GPU:0 with 4057 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5


# Penjelasan
---

##  Catatan Eksekusi Lokal

> **Error**: `TypeError: this __dict__ descriptor does not support '_DictWrapper' objects`  
Error ini terjadi saat menggunakan `tf.saved_model.save()` dan umum ditemukan pada konfigurasi tertentu seperti Conda di WSL. Akibatnya, bagian demonstrasi TensorFlow Serving lokal tidak dapat dijalankan.

---

##  Ringkasan Eksperimen dan Hasil

### 1. TensorFlow Serving (Local Deployment)

- **Tujuan**: Men-serve model dengan latensi rendah dan throughput tinggi melalui REST atau gRPC.
- **Workflow Ideal**:
  - Melatih dan menyimpan model ke `SavedModel`.
  - Menjalankan TFS via Docker, mengawasi direktori `my_mnist_model/{version}`.
  - Mengirim request prediksi ke server.
  - **Fitur Unggulan**: Auto model versioning — model baru pada direktori `0002` otomatis aktif.
- **Hasil**: Tidak berjalan karena error `tf.saved_model.save()`.

>  *Kesimpulan*: Meskipun gagal dieksekusi, TFS tetap merupakan opsi produksi terbaik untuk model serving skala besar.

---

### 2. Google AI Platform (Vertex AI)

- **Tujuan**: Men-deploy model ke cloud managed platform milik Google.
- **Langkah-langkah**:
  - Upload model ke Google Cloud Storage (GCS).
  - Buat objek model dan endpoint di Vertex AI.
  - Deploy model ke endpoint dan uji dengan permintaan prediksi.
- **Hasil**: Hanya dijelaskan secara konseptual, tidak dieksekusi.

>  *Kesimpulan*: Vertex AI sangat cocok untuk perusahaan yang menginginkan skalabilitas, manajemen otomatis, dan integrasi penuh dalam ekosistem Google Cloud.

---

### 3. TensorFlow Lite (Deployment ke Edge/Mobile)

- **Tujuan**: Mengubah model Keras menjadi format ringan `.tflite` untuk perangkat dengan resource terbatas.
- **Langkah Konversi**:
  - `tf.lite.TFLiteConverter.from_keras_model(model)`
  - Simpan ke file `.tflite`
  - Jalankan inferensi menggunakan `tf.lite.Interpreter`
- **Hasil**: Berhasil. Prediksi identik dengan model Keras asli → konversi sukses (lossless).

>  *Kesimpulan*: TFLite adalah solusi solid untuk aplikasi ML di perangkat edge. Mudah dikonversi, cepat dieksekusi, dan hemat daya.

---

##  Perbandingan Metode Deployment

| Metode             | Skenario Utama                | Kelebihan                                                    | Keterbatasan di Notebook                          |
|--------------------|-------------------------------|---------------------------------------------------------------|---------------------------------------------------|
| TensorFlow Serving | Server-side, skala besar       | Throughput tinggi, latensi rendah, auto-versioning            | Gagal disimpan karena environment TypeError       |
| Vertex AI (GCP)    | Cloud-managed deployment       | Skalabilitas mudah, integrasi GCP, tanpa kelola server        | Instruksional, butuh akun GCP dan setup tambahan |
| TensorFlow Lite    | Mobile & edge devices          | Ukuran kecil, cepat, bisa offline, hemat resource             | Berhasil dieksekusi dan tervalidasi               |

---

##  Kesimpulan Akhir

Notebook ini menekankan pentingnya menyesuaikan metode deployment dengan konteks:

- **TFS**: Untuk server dengan kendali penuh dan kebutuhan latensi rendah.
- **Vertex AI**: Untuk deployment cloud skala besar tanpa repot manajemen server.
- **TFLite**: Untuk aplikasi edge/mobile yang efisien dan ringan.